In [ ]:
import pandas as pd
import numpy as np
import openai
from datasets import load_dataset

In [ ]:
openai.api_key = "its a secret heehee"

In [ ]:
data = load_dataset("cs182project/182-final-project")

In [ ]:
data

In [ ]:
df = pd.DataFrame(data['test'])
df

In [ ]:
df['lyrics'] = df['conversations'].map(lambda s: s[1]['value'])
df

In [ ]:
lyric_list = df['lyrics'].to_list()
lyric_list

In [ ]:
input = lyric_list[0:3]
resp = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=input
    )
resp

In [ ]:
embedding_map = dict()
missing = []
for i in range(0, len(lyric_list), 2):
    try:
        input = lyric_list[i:i+2]

        resp = openai.Embedding.create(
            model="text-embedding-ada-002",
            input=input
        )

        data_batch = resp.data

        for data in data_batch:
            embedding_map[i + data.index] = data.embedding
    except Exception as e:
        print(f"ruh roh: {e}")
        missing.append(input)

        


In [ ]:
len(embedding_map)

In [ ]:
missing[0][1]

In [ ]:

resp = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=missing[0][0]
    )
idx = lyric_list.index(missing[0][0])

assert idx not in embedding_map

embedding_map[idx] = resp.data[0].embedding

In [ ]:
df['embedding'] = df['id'].map(lambda i: embedding_map.get(i, np.NaN))
df


In [ ]:
df.dropna()

In [ ]:
df[['id', 'lyrics', 'embedding']].to_parquet("embedding_ground_truth.parquet", index=False)